In [11]:
from keras.layers import Dense,MaxPooling2D,Conv2D,Flatten,BatchNormalization,Input,Dropout
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizer_v1 import adam
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import normalize
import cv2
import numpy as np 
import os 


In [2]:
image_directory='/content/sample_data/data'
noTumor_image_path=os.path.join(image_directory,'no')
Tumor_image_path=os.path.join(image_directory,'yes')
Nameimg_Tumor=os.listdir(Tumor_image_path)
Nameimg_noTumor=os.listdir(noTumor_image_path)

In [3]:
DatasetImg=[]
label=[]
for name in Nameimg_noTumor:
    image=cv2.imread(noTumor_image_path+'/'+name)
    image=cv2.resize(image,(200,200))
    DatasetImg.append(image)
    label.append(0)
for name in Nameimg_Tumor:
    image=cv2.imread(Tumor_image_path+'/'+name)
    image=cv2.resize(image,(200,200))
    DatasetImg.append(image)
    label.append(1)
DatasetImg=np.array(DatasetImg)
label=np.array(label)

In [4]:
train_image,test_image,train_label,test_label=train_test_split(DatasetImg,label,test_size=0.2,random_state=0)

In [5]:
train_image=normalize(train_image,axis=1)
test_image=normalize(test_image,axis=1)

In [10]:
# creat model
input=Input(shape=(200,200,3))
layer=Conv2D(32,(3,3),activation='relu',kernel_initializer='he_uniform')(input)
layer=MaxPooling2D((2,2))(layer)
layer=Conv2D(64,(3,3),activation='relu',kernel_initializer='he_uniform')(layer)
layer=MaxPooling2D((2,2))(layer)
layer=Conv2D(128,(3,3),activation='relu',kernel_initializer='he_uniform')(layer)
layer=MaxPooling2D((2,2))(layer)
flatten=Flatten()(layer)
dense=Dense(128,activation='relu')(flatten)
dense=Dropout(0.2)(dense)
output=Dense(1,activation='sigmoid')(dense)
model=Model(input,output)

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])

In [12]:
checkpoint=ModelCheckpoint('my_model.h5',monitor='val_loss',save_best_only=True)
callbacklist=[checkpoint]

In [13]:
histoty=model.fit(train_image,train_label,batch_size=30,epochs=20,validation_data=(test_image,test_label),
                  callbacks=callbacklist)

Epoch 1/20
83/83 [==============================] - 20s 125ms/step - loss: 0.4728 - acc: 0.7974 - val_loss: 0.2759 - val_acc: 0.8817
Epoch 2/20
83/83 [==============================] - 9s 110ms/step - loss: 0.1733 - acc: 0.9331 - val_loss: 0.1647 - val_acc: 0.9400
Epoch 3/20
83/83 [==============================] - 9s 109ms/step - loss: 0.0798 - acc: 0.9733 - val_loss: 0.1360 - val_acc: 0.9643
Epoch 4/20
83/83 [==============================] - 9s 105ms/step - loss: 0.0336 - acc: 0.9911 - val_loss: 0.1635 - val_acc: 0.9530
Epoch 5/20
83/83 [==============================] - 9s 109ms/step - loss: 0.0222 - acc: 0.9911 - val_loss: 0.1265 - val_acc: 0.9643
Epoch 6/20
83/83 [==============================] - 9s 110ms/step - loss: 0.0088 - acc: 0.9992 - val_loss: 0.1061 - val_acc: 0.9724
Epoch 7/20
83/83 [==============================] - 9s 107ms/step - loss: 0.0039 - acc: 0.9992 - val_loss: 0.1502 - val_acc: 0.9692
Epoch 8/20
83/83 [==============================] - 9s 107ms/step - loss: 0

In [15]:
model.save('/content/sample_data/data.mymodel.h5')